# Random Search Example

In this example we will perform a RandomSearch using Scikit-learn and Keras on the [breast cancer](https://github.com/autonomio/datasets/blob/master/autonomio-datasets/breast_cancer.csv) classification task. You can run this example on CPU. It will take more or less 5 minutes.

We will continue to use the same example of the [Grid Search](./grid_search_example.ipynb) notebook, but we will search for a different subset of hyperparameters.

## Initial Setup

Import the packages we need for the computation.

In [1]:
import os
import pandas as pd
import wrangle as wr

from numpy import nan

from scipy.stats import randint as sp_randint
from scipy.stats import uniform

from keras.utils import to_categorical
from keras.wrappers.scikit_learn import KerasClassifier

# Mounting point
MP = '/floyd/input/bcds'

Using TensorFlow backend.


### Load the dataset

Load, clean and preprocess the dataset

In [2]:
def breast_cancer():
    '''Load and preprocess(cleaning) the dataset'''
    df = pd.read_csv(os.path.join(MP, 'breast_cancer.csv'))
    
    # then some minimal data cleanup
    df.drop("Unnamed: 32", axis=1, inplace=True)
    df.drop("id", axis=1, inplace=True)

    # separate to x and y
    y = df.diagnosis.values
    x = df.drop('diagnosis', axis=1).values

    # convert the string labels to binary
    y = (y == 'M').astype(int)

    return x, y

In [3]:
# Load the dataset
x, y = breast_cancer()

# Normalize every feature to mean 0, std 1
x = wr.mean_zero(pd.DataFrame(x)).values

input_dim = x.shape[1] # number of columns

### Model definition

Define the model and the variables to search.

In [4]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras import optimizers

# Function to create model, required for KerasClassifier
def create_model(first_neuron=9,
                 activation='relu',
                 kernel_initializer='uniform',
                 dropout_rate=0,
                 optimizer='adam'):
    
    # Create model
    model = Sequential()
    # L1
    model.add(Dense(first_neuron, 
                    input_dim=input_dim, 
                    kernel_initializer=kernel_initializer, 
                    activation=activation))
    # Dropout
    model.add(Dropout(dropout_rate))
    # L2
    model.add(Dense(1, kernel_initializer=kernel_initializer, activation='sigmoid'))
    
    # Compile model
    model.compile(loss='binary_crossentropy', 
                  optimizer=optimizer, 
                  metrics=['accuracy'])
    return model

In [5]:
# Create the model
model = KerasClassifier(build_fn=create_model) 

### Range of Values - The Grid

Defining the parameter space boundaries.

In [6]:
# Specify parameters and distributions to sample from

# Model Design Components
first_neurons = sp_randint(8, 15)
activation =  ['relu', 'elu'] # You can also try 'tanh', 'sigmoid', 'hard_sigmoid', 'linear'
kernel_initializer = ['uniform', 'normal'] # You can also try lecun_uniform', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform'
optimizer= ['adam', 'nadam', 'sgd']

# Hyperparameters
epochs = [10] # You can also try 20, 30, 40, etc...
batch_size = [1024] # You can also try 2, 4, 8, 16, 32, 64, 128 etc...
dropout_rate = uniform(0.0, 1) # Sample a value of dropout in the range [0, 0.5]

In [7]:
# Prepare for the Search
param_dist = dict(epochs=epochs, 
                  batch_size=batch_size, 
                  optimizer=optimizer,
                  dropout_rate=dropout_rate,
                  activation=activation,
                  kernel_initializer=kernel_initializer,
                  first_neuron=first_neurons)

### Grid Search

Run the search: 3 folds cross-validation (`cv=3`) on a single process (`n_jobs=1`).

In [8]:
# Perform the Search!
from sklearn.model_selection import RandomizedSearchCV

# run randomized search
n_iter_search = 8 # Use half number of iterations performed by Grid Search
random_search = RandomizedSearchCV(estimator=model, 
                                   param_distributions=param_dist,
                                   n_iter=n_iter_search,
                                   n_jobs=1, cv=3, verbose=2)
random_search.fit(x, y)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] activation=elu, batch_size=1024, dropout_rate=0.6010381533043048, epochs=10, first_neuron=10, kernel_initializer=normal, optimizer=adam 
Epoch 1/10
379/379 [==============================] - 0s 877us/step - loss: 0.6857 - acc: 0.6385
Epoch 2/10
379/379 [==============================] - 0s 11us/step - loss: 0.6849 - acc: 0.6016
Epoch 3/10
379/379 [==============================] - 0s 6us/step - loss: 0.6776 - acc: 0.7098
Epoch 4/10
379/379 [==============================] - 0s 7us/step - loss: 0.6761 - acc: 0.7230
Epoch 5/10
379/379 [==============================] - 0s 8us/step - loss: 0.6727 - acc: 0.7388
Epoch 6/10
379/379 [==============================] - 0s 7us/step - loss: 0.6697 - acc: 0.8047
Epoch 7/10
379/379 [==============================] - 0s 7us/step - loss: 0.6667 - acc: 0.8047
Epoch 8/10
379/379 [==============================] - 0s 8us/step - loss: 0.6629 - acc: 0.8338
Epoch 9/10
379/379 [===============

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.0s remaining:    0.0s


Epoch 1/10
379/379 [==============================] - 0s 650us/step - loss: 0.7021 - acc: 0.3773
Epoch 2/10
379/379 [==============================] - 0s 12us/step - loss: 0.6987 - acc: 0.4327
Epoch 3/10
379/379 [==============================] - 0s 11us/step - loss: 0.6939 - acc: 0.4538
Epoch 4/10
379/379 [==============================] - 0s 13us/step - loss: 0.6944 - acc: 0.5145
Epoch 5/10
379/379 [==============================] - 0s 5us/step - loss: 0.6879 - acc: 0.6095
Epoch 6/10
379/379 [==============================] - 0s 11us/step - loss: 0.6848 - acc: 0.6253
Epoch 7/10
379/379 [==============================] - 0s 5us/step - loss: 0.6844 - acc: 0.6412
Epoch 8/10
379/379 [==============================] - 0s 4us/step - loss: 0.6787 - acc: 0.6992
Epoch 9/10
379/379 [==============================] - 0s 6us/step - loss: 0.6726 - acc: 0.7573
Epoch 10/10
379/379 [==============================] - 0s 10us/step
[CV]  activation=elu, batch_size=1024, dropout_rate=0.6010381533043048,

[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed:   26.3s finished


Epoch 1/10
569/569 [==============================] - 1s 1ms/step - loss: 0.6953 - acc: 0.4728
Epoch 2/10
569/569 [==============================] - 0s 4us/step - loss: 0.6928 - acc: 0.5009
Epoch 3/10
569/569 [==============================] - 0s 4us/step - loss: 0.6909 - acc: 0.5870
Epoch 4/10
569/569 [==============================] - 0s 4us/step - loss: 0.6880 - acc: 0.6028
Epoch 5/10
569/569 [==============================] - 0s 4us/step - loss: 0.6845 - acc: 0.6538
Epoch 6/10
569/569 [==============================] - 0s 4us/step - loss: 0.6805 - acc: 0.7030
Epoch 7/10
569/569 [==============================] - 0s 4us/step - loss: 0.6787 - acc: 0.7557
Epoch 8/10
569/569 [==============================] - 0s 4us/step - loss: 0.6763 - acc: 0.7399
Epoch 9/10
569/569 [==============================] - 0s 4us/step - loss: 0.6734 - acc: 0.7610
Epoch 10/10
569/569 [==============================] - 0s 4us/step - loss: 0.6702 - acc: 0.7979


RandomizedSearchCV(cv=3, error_score='raise',
          estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x7f79ec3c3c50>,
          fit_params=None, iid=True, n_iter=8, n_jobs=1,
          param_distributions={'epochs': [10], 'batch_size': [1024], 'optimizer': ['adam', 'nadam', 'sgd'], 'dropout_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f79ec360ac8>, 'activation': ['relu', 'elu'], 'kernel_initializer': ['uniform', 'normal'], 'first_neuron': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f79ec3609b0>},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, verbose=2)

### Results

Let's see which configuration give us the best performance.

*Note*: it's likely that the Random Search we will perform slightly worse than Random Search in this toy example.

In [9]:
# Show results
print("Best: %f using %s" % (random_search.best_score_, random_search.best_params_))
means = random_search.cv_results_['mean_test_score']
stds = random_search.cv_results_['std_test_score']
params = random_search.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.894552 using {'activation': 'elu', 'batch_size': 1024, 'dropout_rate': 0.6010381533043048, 'epochs': 10, 'first_neuron': 10, 'kernel_initializer': 'normal', 'optimizer': 'adam'}
0.894552 (0.034227) with: {'activation': 'elu', 'batch_size': 1024, 'dropout_rate': 0.6010381533043048, 'epochs': 10, 'first_neuron': 10, 'kernel_initializer': 'normal', 'optimizer': 'adam'}
0.806678 (0.184844) with: {'activation': 'relu', 'batch_size': 1024, 'dropout_rate': 0.39983996327340543, 'epochs': 10, 'first_neuron': 8, 'kernel_initializer': 'uniform', 'optimizer': 'adam'}
0.843585 (0.077833) with: {'activation': 'relu', 'batch_size': 1024, 'dropout_rate': 0.7129907024046704, 'epochs': 10, 'first_neuron': 13, 'kernel_initializer': 'normal', 'optimizer': 'adam'}
0.581722 (0.065330) with: {'activation': 'relu', 'batch_size': 1024, 'dropout_rate': 0.3072453873479509, 'epochs': 10, 'first_neuron': 10, 'kernel_initializer': 'uniform', 'optimizer': 'sgd'}
0.773286 (0.112224) with: {'activation': 'elu'

**That's all folks - don't forget to shutdown your workspace once you're done 🙂**